In [2]:
import pandas as pd
import numpy as np
import datetime
import math

In [3]:
# Loading all 12 months files and appending df_list to consolidate
file_names = [f"2023{count:02d}-divvy-tripdata.csv" for count in range(1, 13)]
df_list = [pd.read_csv(file_name) for file_name in file_names]

# Consolidate the whole data into one dataframe
df = pd.concat(df_list, ignore_index = True)

print(f"Total record counts before cleaning = {df.shape[0]}")

Total record counts before cleaning = 5719877


In [4]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,Lincoln Ave & Fullerton Ave,TA1309000058,Hampden Ct & Diversey Ave,202480.0,41.924074,-87.646278,41.930000,-87.640000,member
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008571,-87.690483,42.039742,-87.699413,casual
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member


In [5]:
# Datatype of columns
print(df.dtypes)

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object


In [6]:
# Parsing started_at and ended_at columns
df["ended_at"] = pd.to_datetime(df["ended_at"], format = "%Y-%m-%d %H:%M:%S")
df["started_at"] = pd.to_datetime(df["started_at"], format = "%Y-%m-%d %H:%M:%S")

In [7]:
# Adding ride_duration column
df["ride_duration"] = (df["ended_at"] - df["started_at"]).dt.total_seconds()

In [8]:
# Adding ride_started_on_day column
df["ride_started_on_day"] = df["started_at"].dt.day_name()
df["ride_ended_on_day"] = df["ended_at"].dt.day_name()

In [9]:
# Concating lat and lng of both start and end station
df["start_cord"] = df["start_lat"].astype(str) + ", " + df["start_lng"].astype(str)
df["end_cord"] = df["end_lat"].astype(str) + ", " + df["end_lng"].astype(str)

In [10]:
# Eliminating rows in which start and end station lat and lng are missing
df = df[~((pd.isna(df["end_lat"])) & (pd.isna(df["end_lng"])))]
df = df[~((pd.isna(df["start_lat"])) & (pd.isna(df["start_lng"])))]

In [11]:
# Eliminating rows having start and end station coordinates (start_lat = 0.0 and start_lng = 0.0)
df = df[~(df["start_cord"] == "0.0, 0.0")]
df = df[~(df["end_cord"] == "0.0, 0.0")]

In [12]:
# Finding displacement between starting and ending station (Haversine formula)
earth_radius=6371
lat1, lon1, lat2, lon2 = np.radians([df["start_lat"], df["start_lng"], df["end_lat"], df["end_lng"]])
a = np.sin((lat2 - lat1) / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0) ** 2
df["station_displacement"] = earth_radius * 2 * np.arcsin(np.sqrt(a))

In [13]:
# Eliminating rows that having ride duration is negative or zero second.
df = df[~(df["started_at"] >= df["ended_at"])]

# Eliminating rows that having ride duration greater than 1 day (considering outliers)
df = df[~(df["ride_duration"] > (24 * 60 * 60))]

print(f"Total record counts after cleaning = {df.shape[0]}")

Total record counts after cleaning = 5711396


In [14]:
# Datatype of columns
print(df.dtypes)

ride_id                         object
rideable_type                   object
started_at              datetime64[ns]
ended_at                datetime64[ns]
start_station_name              object
start_station_id                object
end_station_name                object
end_station_id                  object
start_lat                      float64
start_lng                      float64
end_lat                        float64
end_lng                        float64
member_casual                   object
ride_duration                  float64
ride_started_on_day             object
ride_ended_on_day               object
start_cord                      object
end_cord                        object
station_displacement           float64
dtype: object


In [15]:
df.describe()

,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,ride_duration,station_displacement
count,5711396,5711396,5.711396e+06,5.711396e+06,5.711396e+06,5.711396e+06,5.711396e+06,5.711396e+06
mean,2023-07-16 10:01:50.532310272,2023-07-16 10:16:56.462769664,4.190290e+01,-8.764704e+01,4.190324e+01,-8.764725e+01,9.059305e+02,2.097852e+00
min,2023-01-01 00:01:58,2023-01-01 00:02:41,4.163000e+01,-8.794000e+01,4.161000e+01,-8.816000e+01,1.000000e+00,0.000000e+00
25%,2023-05-21 12:31:01.750000128,2023-05-21 12:52:45.249999872,4.188096e+01,-8.766000e+01,4.188103e+01,-8.766027e+01,3.250000e+02,8.625573e-01
50%,2023-07-20 17:50:18,2023-07-20 18:05:23.500000,4.189908e+01,-8.764404e+01,4.190000e+01,-8.764410e+01,5.710000e+02,1.540571e+00
75%,2023-09-16 20:11:54,2023-09-16 20:28:41.500000,4.193000e+01,-8.762991e+01,4.193000e+01,-8.763000e+01,1.013000e+03,2.747891e+00
max,2023-12-31 23:59:38,2024-01-01 14:20:23,4.207000e+01,-8.746000e+01,4.218000e+01,-8.744000e+01,8.639200e+04,4.892880e+01
std,NaN,NaN,4.503617e-02,2.732708e-02,4.518724e-02,2.745879e-02,1.849784e+03,1.901604e+00


In [222]:
# Creating a colsolidated csv file for visualization purpose
df.to_csv("Cleaned_bike_sharing_data.csv", index = False)

In [16]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'ride_duration', 'ride_started_on_day',
       'ride_ended_on_day', 'start_cord', 'end_cord', 'station_displacement'],
      dtype='object')

In [28]:
# Different counts
casual_count = df[df["member_casual"] == "casual"].shape[0]
member_count = df[df["member_casual"] == "member"].shape[0]
total_count = df.shape[0]

count_dict = {
    "Casual" : [casual_count, (casual_count / total_count) * 100],
    "Member" : [member_count, (member_count / total_count) * 100],
    "Total" : [total_count, None]
}

count_df = pd.DataFrame(count_dict, columns = ["Casual", "Member", "Total"], index = ["Count", "% of total"])
print(count_df)

                  Casual        Member      Total
Count       2.052585e+06  3.658811e+06  5711396.0
% of total  3.593841e+01  6.406159e+01        NaN


In [26]:
# Finding relationship between bike type and rider type
bike_type_df = df.groupby(["member_casual", "rideable_type"]).size().unstack(fill_value = 0)
print(bike_type_df)

rideable_type  classic_bike  docked_bike  electric_bike
member_casual                                          
casual               872968        76088        1103529
member              1817851            0        1840960


In [36]:
# Finding the relationship between number of rides based on month for rider type
month_ride_df = df.groupby(["member_casual", pd.Grouper(key = "started_at", freq = "M")]).size().unstack(fill_value = 0)
month_ride_df.columns = month_ride_df.columns.strftime('%b')
month_ride_df

started_at,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
member_casual,,,,,,,,,,,,
casual,39899,42919,62040,146868,233549,300383,330123,309907,260819,176540,98060,51478
member,150258,147397,196432,279241,370483,418245,436036,460238,404473,359802,263929,172277


Findings:
1. Both the riders are taking rides during summer season i.e. during the months of June, July, and August.
2. Casual rides are having maximum rides in July Month and member riders are having in August month

In [24]:
# Finding the relationship between average ride duration based on month for member type
ride_duration_df = df.groupby(["member_casual", pd.Grouper(key = "started_at", freq = "M")])["ride_duration"].mean().unstack()
ride_duration_df.columns = ride_duration_df.columns.strftime("%b")
print(ride_duration_df)

started_at            Jan         Feb         Mar          Apr          May  \
member_casual                                                                 
casual         808.874007  954.086256  908.754691  1217.602568  1312.126838   
member         603.915379  627.769202  611.542967   687.607443   753.662673   

started_at             Jun          Jul          Aug          Sep  \
member_casual                                                       
casual         1292.877210  1358.199498  1309.688932  1269.176617   
member          770.953895   791.564135   789.720140   757.171566   

started_at             Oct         Nov         Dec  
member_casual                                       
casual         1138.352424  953.146930  879.032577  
member          693.254176  656.479868  646.023509  


Findings:
1. Casual riders take longer rides compared to member riders irrespective of month and season.
2. Both the riders are taking longer rides in summer season i.e. months of June, July, and August.

In [32]:
# Finding the relationship between number of rides based on day of week for member type
day_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
day_rides_df = df.groupby(["member_casual", "ride_started_on_day"]).size().unstack(fill_value = 0)
day_rides_df = day_rides_df[day_of_week]
day_rides_df

ride_started_on_day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
member_casual,,,,,,,
casual,234117,245529,248479,269827,310965,409275,334393
member,494328,576459,586189,589308,531327,472588,408612


Finding: Casual members are taking more rides on weekends and annual members are taking during week days more often.

In [35]:
# Finding the relationship between average ride duration based on day of week for member type
day_ride_duration_df = df.groupby(["member_casual", "ride_started_on_day"])["ride_duration"].mean().unstack(fill_value = 0)
day_ride_duration_df = day_ride_duration_df[day_of_week]
day_ride_duration_df

ride_started_on_day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
member_casual,,,,,,,
casual,1213.114763,1102.218158,1054.857292,1074.727492,1195.143470,1393.485087,1433.908739
member,686.967095,694.860837,690.448253,694.489651,719.285696,804.440362,806.247408


Finding: 
1. Both riders takes longer rides over weekends.
2. Casual riders take longer rides compared to member riders irrespective of day of week.

In [42]:
# Finding the relationship between number of rides based on hour of the day for rider type
hourly_ride_df = df.groupby(["member_casual", df["started_at"].dt.hour]).size().unstack(fill_value = 0)
hourly_ride_df

started_at,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
member_casual,,,,,,,,,,,,,,,,,,,,,
casual,36700,23777,14358,7880,5921,11400,30085,52919,70547,69790,...,142277,158743,181950,198739,171607,126843,91627,77070,68134,49084
member,35505,21145,12256,7925,8778,34131,105282,194535,244117,164831,...,201806,246625,331465,387753,307627,217844,151590,117706,87963,56396


In [46]:
# Finding the relationship between returning bicycle to same station to member type
member = df[(df["start_cord"] == df["end_cord"]) & (df["member_casual"] == "member")].shape[0]
casual = df[(df["start_cord"] == df["end_cord"]) & (df["member_casual"] == "casual")].shape[0]

return_to_same_station = {
    "member" : [member, (member / df[(df["member_casual"] == "member")].shape[0]) * 100],
    "casual" : [casual, (casual / df[(df["member_casual"] == "casual")].shape[0]) * 100] 
}
return_to_same_station_df = pd.DataFrame(return_to_same_station, index = ["Count", "% of rider type"])
return_to_same_station_df

,member,casual
Count,131999.000000,155757.000000
% of rider type,3.607702,7.588334
